In [ ]:
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install scikit-learn pillow torchvision opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install datasets

from sklearn.model_selection import train_test_split
from datasets import load_dataset, DatasetDict
import evaluate

In [ ]:
from datasets import load_dataset

ds = load_dataset("uoft-cs/cifar10")

README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
ds = ds['train'].train_test_split(test_size=0.3, stratify_by_column="label")  # 70% train, 30% test
ds_test = ds['test'].train_test_split(test_size=0.5, stratify_by_column="label")  # 30% test --> 15% valid, 15% test
ds = DatasetDict({
    'train': ds['train'],
    'test': ds_test['test'],
    'valid': ds_test['train']
})

del ds_test

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 7500
    })
    valid: Dataset({
        features: ['img', 'label'],
        num_rows: 7500
    })
})

In [ ]:
labels = ds['train'].features['label']
labels

ClassLabel(names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], id=None)

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

def transform(example_batch):
    # Define the desired image size
    desired_size = (224, 224)

    # Resize the images in the 'img' column to the desired size
    resized_images = [transforms.Resize(desired_size)(x.convert("RGB")) for x in example_batch['img']]

    # Convert resized images to pixel values using the processor
    inputs = processor(resized_images, return_tensors='pt')

    # Include the labels
    inputs['labels'] = example_batch['label']

    return inputs

# Apply the transform function to the dataset
prepared_ds = ds.with_transform(transform)


In [ ]:
import numpy as np
import torch

# Define collate function
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])  # Use 'labels' instead of 'label'
    }


# Load the accuracy metric using the evaluate library
metric = evaluate.load("accuracy")

# Define the compute_metrics function
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification

model_name_or_path = 'google/vit-base-patch16-224-in21k'

# Ensure labels is a list of label names
if hasattr(labels, 'names'):
    labels = labels.names  # Convert ClassLabel object to a list of names if necessary

processor = AutoImageProcessor.from_pretrained(model_name_or_path)
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True
)


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

root_dir = "./ViT_custom"  # Set root directory within Colab's environment

training_args = TrainingArguments(
    output_dir=root_dir,
    per_device_train_batch_size=4,  # Smaller batch size to fit in limited memory
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,  # Enables mixed precision if GPU supports it; set to False if unstable
    num_train_epochs=1,
    logging_steps=1000,  # Log less frequently
    learning_rate=1e-4,  # Lower learning rate for small batch size
    gradient_accumulation_steps=2,  # Accumulate gradients to simulate larger batch size
    save_total_limit=1,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["valid"],
    tokenizer=processor,
)

<ipython-input-14-a4fb768c0cda>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

print(ds.column_names)


{'train': ['img', 'label'], 'test': ['img', 'label'], 'valid': ['img', 'label']}


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, recall_score

test_ds = ds['test'].with_transform(transform)
test_outputs = trainer.predict(test_ds)

y_true = test_outputs.label_ids
y_pred = test_outputs.predictions.argmax(1)

labels = test_ds.features["label"].names
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(xticks_rotation=45)

recall = recall_score(y_true, y_pred, average=None)

# Print the recall for each class
for label, score in zip(labels, recall):
  print(f"Recall for {label}: {score:.2f}")


In [ ]:
def getPrediction(image):
    model_name_or_path = 'google/vit-base-patch16-224-in21k'
    processor = AutoImageProcessor.from_pretrained(model_name_or_path)
    vit = ViTForImageClassification.from_pretrained(save_dir)
    model = pipeline('image-classification', model=vit, feature_extractor=processor)

    result = model(image)
    return result

In [ ]:
import cv2

image = cv2.imread(".../path/to/image.jpg")
print(getPrediction(image))

In [ ]:
from datasets import load_dataset, DatasetDict
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import torch
from transformers import AutoImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
import evaluate
import os

# Load CIFAR-10 dataset and take a subset for quick execution
ds = load_dataset("uoft-cs/cifar10")
ds = ds['train'].train_test_split(test_size=0.3, stratify_by_column="label")

# Take a smaller subset for faster execution
ds = DatasetDict({
    'train': ds['train'].select(range(1000)),  # Use first 1000 samples for training
    'valid': ds['test'].select(range(500)),    # Use first 500 samples for validation
})

def transform(example_batch):
    # Desired image size
    desired_size = (224, 224)
    # Resize images in 'img' column to the desired size
    resized_images = [transforms.Resize(desired_size)(x) for x in example_batch['img']]
    # Convert resized images to pixel values using processor
    inputs = processor(resized_images, return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs


# Apply transform to dataset
prepared_ds = ds.with_transform(transform)

# Define collate function
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

# Load the accuracy metric
metric = evaluate.load("accuracy")

# Define compute_metrics function
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

# Load Model and Processor
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = AutoImageProcessor.from_pretrained(model_name_or_path)
labels = ds['train'].features['label'].names
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: str(i) for i, label in enumerate(labels)},
    ignore_mismatched_sizes=True
)

# Training Arguments with Reduced Logging and Higher Learning Rate
root_dir = "./ViT_custom"  # Directory within Colab's environment
os.makedirs(root_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=root_dir,
    per_device_train_batch_size=4,
    evaluation_strategy="no",  # Disable evaluation to speed up training
    save_strategy="no",        # Disable saving intermediate steps
    fp16=True,
    num_train_epochs=1,        # Only 1 epoch
    learning_rate=2e-3,        # Higher learning rate for quick convergence
    logging_steps=5000,        # Log less frequently
    gradient_accumulation_steps=1,  # Remove gradient accumulation
    save_total_limit=1,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='none',          # Disable TensorBoard logging
    load_best_model_at_end=False,
)

# Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["valid"],
    tokenizer=processor,
)

# Train the Model
train_results = trainer.train()

# Save the Model
save_dir = f"{root_dir}/best_model"
trainer.save_model(save_dir)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-1485b03ad259>:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification, pipeline
from PIL import Image
import requests
from io import BytesIO

# Path to your fine-tuned model
save_dir = './ViT_custom/best_model'

# Define the prediction function with enhanced preprocessing
def getPrediction(image):
    # Load the processor and fine-tuned model
    processor = AutoImageProcessor.from_pretrained(save_dir)
    vit = ViTForImageClassification.from_pretrained(save_dir)
    # Initialize the pipeline with your fine-tuned model, specifying device=0 for GPU
    model = pipeline('image-classification', model=vit, feature_extractor=processor, device=0)

    # Resize the image to the expected input size for CIFAR-10 (32x32 pixels)
    image = image.resize((32, 32))

    # Run the model on the input image and get top-5 predictions
    result = model(image, top_k=5)
    return result

# URL of the image to test
image_url = "https://images.unsplash.com/photo-1551963831-b3b1ca40c98e"

# Fetch the image from the URL
response = requests.get(image_url)
img_data = BytesIO(response.content)

# Load the image with PIL and convert to RGB
pil_image = Image.open(img_data).convert("RGB")

# Get and print top-5 predictions
print(getPrediction(pil_image))


[{'label': 'frog', 'score': 0.7378625273704529}, {'label': 'deer', 'score': 0.6351392269134521}, {'label': 'dog', 'score': 0.5646997690200806}, {'label': 'bird', 'score': 0.5600773692131042}, {'label': 'truck', 'score': 0.5119665265083313}]


In [ ]:
from datasets import load_dataset, DatasetDict
from PIL import Image
import torchvision.transforms as transforms
import torch

# Load the CIFAR-10 dataset and split into train/valid/test
ds = load_dataset("uoft-cs/cifar10")
ds = ds['train'].train_test_split(test_size=0.2, stratify_by_column="label")
ds_test = ds['test'].train_test_split(test_size=0.5, stratify_by_column="label")
ds = DatasetDict({
    'train': ds['train'],
    'valid': ds_test['train'],
    'test': ds_test['test']
})

# Define transformation with mild data augmentation
def transform(example_batch):
    # Data augmentation
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
    ])
    images = [transform(Image.fromarray(img)) for img in example_batch['img']]  # Use 'img' column
    inputs = processor(images, return_tensors="pt")
    inputs['labels'] = example_batch['label']
    return inputs

# Apply the transform to the dataset
prepared_ds = ds.with_transform(transform)



In [ ]:
print(ds.column_names)


{'train': ['img', 'label'], 'valid': ['img', 'label'], 'test': ['img', 'label']}


In [ ]:
from transformers import TrainingArguments

root_dir = "./ViT_custom"
training_args = TrainingArguments(
    output_dir=root_dir,
    per_device_train_batch_size=16,         # Higher batch size if possible
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,                              # Mixed precision
    num_train_epochs=5,                     # Set max epochs, but we’ll use early stopping
    learning_rate=2e-4,                     # Start with a moderately high learning rate
    lr_scheduler_type="cosine",             # Cosine scheduler for gradual learning rate decay
    logging_steps=500,
    save_total_limit=2,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification, Trainer
import evaluate
import numpy as np

# Load processor and model
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = AutoImageProcessor.from_pretrained(model_name_or_path)
labels = ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: str(i) for i, label in enumerate(labels)},
)

# Define accuracy as the evaluation metric
metric = evaluate.load("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=lambda data: {
        'pixel_values': torch.stack([f['pixel_values'] for f in data]),
        'labels': torch.tensor([f['labels'] for f in data])
    },
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds['train'],
    eval_dataset=prepared_ds['valid'],
    tokenizer=processor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)


<ipython-input-21-0a56c734d23e>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
train_results = trainer.train()
save_dir = f"{root_dir}/best_model"
trainer.save_model(save_dir)
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()


KeyError: 'img'

In [ ]:
from datasets import load_dataset, DatasetDict
from PIL import Image
import torchvision.transforms as transforms
from transformers import AutoImageProcessor, ViTForImageClassification, TrainingArguments, Trainer
import evaluate
import torch
import numpy as np

# Load dataset
ds = load_dataset("uoft-cs/cifar10")
ds = ds['train'].train_test_split(test_size=0.2, stratify_by_column="label")
ds_test = ds['test'].train_test_split(test_size=0.5, stratify_by_column="label")
ds = DatasetDict({
    'train': ds['train'],
    'valid': ds_test['train'],
    'test': ds_test['test']
})

# Define transformation function
def preprocess_function(example):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
    ])
    # Apply transformations, handling both array and PIL Image cases
    if isinstance(example['img'], Image.Image):
        # If 'img' is already a PIL Image
        image = example['img']
    else:
        # Otherwise, convert from array
        image = Image.fromarray(example['img'])

    example['pixel_values'] = transform(image)
    return example


# Apply transformations to each dataset split
prepared_ds = ds.map(preprocess_function, remove_columns=['img'])

# Load processor and model
model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = AutoImageProcessor.from_pretrained(model_name_or_path, use_fast=True)
labels = ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: str(i) for i, label in enumerate(labels)},
    ignore_mismatched_sizes=True
)

# Define accuracy metric
metric = evaluate.load("accuracy")

def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

# Define training arguments
root_dir = "./ViT_custom"
training_args = TrainingArguments(
    output_dir=root_dir,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    num_train_epochs=3,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    logging_steps=500,
    save_total_limit=2,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

# Initialize Trainer without collate_fn (already preprocessed)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=prepared_ds['train'],
    eval_dataset=prepared_ds['valid'],
    compute_metrics=compute_metrics,
)

# Train the model
train_results = trainer.train()

# Save the model
save_dir = f"{root_dir}/best_model"
trainer.save_model(save_dir)
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()
